In [3]:
import pandas as pd
import numpy as np

# Visualization library
import altair as alt
alt.data_transformers.enable('default', max_rows=None)

# Dates management
import datetime

# For the computation of Kaplan-Meier estimates and log-rank tests
import lifelines

#Utiliser edsnlp pour extraitre les 3 entités ci dessus : 
import spacy
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# Load declared pipelines
# from edsnlp import components
from edsnlp.processing.parallel import pipe as parallel_pipe
from spacy import displacy

c:\Users\gazze\anaconda3\envs\eds-tuto\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
# Patients
from dedup_proba import df_person_dedup_proba as df_person
# Visits
from dedup_proba import df_visit_dedup_proba as df_visit
# Diagnosis (condition)
from dedup_proba import df_condition_dedup_proba as df_condition
# Cancer patients
from dedup_proba import df_cancer_dedup_proba as df_cancer
# nbre de patients de cancer
from dedup_proba import nbre_patients_cancer_dedup_proba as nbre_patients_cancer
# risk factors
from dedup_proba import risk_factors

ImportError: cannot import name 'risk_factors' from 'dedup_proba' (c:\Users\gazze\Desktop\EI EL-APHP\ei-aphp\dedup_proba.py)

In [21]:
print(cancer_condition)

{'C503', 'C504', 'C500', 'C502', 'C501', 'C50'}


In [5]:
df_note = pd.read_pickle('data/df_note.pkl')

In [6]:
# Clinicians indicated that the following synonyms could be used

terms = dict(
   tabac=['tabac','fumeur','tabagisme','nicotine','cigarette','cigarettes','fume','fumer','fumait','tabagique','fumeuse','fumé'],
   alcool=['alcool','alcoolisme','Alcoolodépendance','Alcoolodépendant','Alcoolisation','alcoolique'],
   diabete=['Diabète','Hyperglycémie','Hypoglycémie','Diabétique','Polyurie','Polydipsie','Polyphagie','diabète','hyperglycémie','hypoglycémie','diabétique','polyurie','polydipsie','polyphagie'],
   sub_psy=['Psychoactive','Psychotrope','Psychostimulant','Psychodysleptique','Psychodélirant','Narcotique','Sédatif','Hypnotique','Anxiolytique','Antidépresseur','Stimulant','Hallucinogène','Dépresseur','Opiacé','Cannabinoïde','Dissociatif','Inhalant','psychoactive','psychotrope','psychostimulant','psychodysleptique','psychodélirant','narcotique','sédatif','hypnotique','anxiolytique','antidépresseur','stimulant','hallucinogène','dépresseur','opiacé','cannabinoïde','dissociatif','inhalant'],
   tum_herit=['antécédents familiaux de tumeur maligne du sein']
)

In [8]:
def pick_results(doc):
    """
    This function provides the entities that must be collected by the nlp process.
    """
    return [{
             'note_id':e.doc._.note_id,
             'visit_occurrence_id':e.doc._.visit_occurrence_id,
             'lexical_variant':e.text,
             'label':e.label_,
             'negation':e._.negation
             } 
             for e in doc.ents if doc.ents]

In [10]:
def calc_value(x):
    state = {'tabac':'unknown', 'alcool':'unknown', 'diabete':'unknown', 'sub_psy':'unknown', 'tum_herit':'unknown', }
    for _, row in x.iterrows():
        for factor in state.keys():
            if row['label'] == factor and row['negation']:
                if state[factor] == "unknown" :
                    state[factor] = "False"
                elif state[factor] == "True" :
                    state[factor] = "ambiguous"
            if row['label'] == factor and not(row['negation']):
                if state[factor] == "unknown" :
                    state[factor] = "True"
                elif state[factor] == "False" :
                    state[factor] = "ambiguous"
    return state

In [7]:
nlp = spacy.blank("fr")
# sentencizer component
nlp.add_pipe('eds.sentences')
nlp.add_pipe("eds.normalizer")
# Matcher component
nlp.add_pipe("eds.matcher", config=dict(terms=terms))
nlp.add_pipe("eds.negation")

In [ ]:
ents = parallel_pipe(
                df_note,
                nlp,
                context=['note_id', 'visit_occurrence_id'],
                progress_bar=False,
                n_jobs=1, 
                results_extractor = pick_results,
                )

In [ ]:
ents_grouped = ents.groupby('visit_occurrence_id').apply(calc_value).to_frame('state').reset_index()
for factor in 
    ents_grouped['tabac'] = ents_grouped.state.apply(lambda x : x['tabac'])
